In [61]:
import openai 
import pandas as pd 
import numpy as np
import json
from datetime import datetime
from dfply import *
from plotnine import *

In [114]:
conversations = pd.read_csv("cornell-dialog-data/movie_conversations.tsv", encoding='utf-8-sig', sep="\t", header = None)
titles = pd.read_csv("cornell-dialog-data/movie_titles_metadata.tsv", encoding='utf-8-sig', sep="\t", header = None)
lines_raw = pd.read_csv("cornell-dialog-data/movie_lines.tsv", encoding='utf-8-sig', header = None)
lines_list = lines_raw[0].str.split('\t')
characters_raw = pd.read_csv("cornell-dialog-data/movie_characters_metadata.tsv", encoding='utf-8-sig', header = None)
characters_list = characters_raw[0].str.split('\t')


In [115]:
lines = []
for i in range(0, len(lines_list)):
    v = lines_list[i]

    if len(v) <= 5:
        lines.append(v)

characters = []
for i in range(0, len(characters_list)):
    v = characters_list[i]

    if len(v) <= 6:
        characters.append(v)

lines = pd.DataFrame(lines)
characters = pd.DataFrame(characters)

In [116]:
conversations.columns = conversations.columns.map(str)
lines.columns = lines.columns.map(str)
titles.columns = titles.columns.map(str)
characters.columns = characters.columns.map(str)

In [148]:
#cleaning
lines['line_number'] = [int(x[1:]) for x in lines['0']]
lines = lines.sort_values(by = ['2', 'line_number'])

lines['next_movie'] = lines['2'].shift(1)

lines['line_number_shifted'] = lines.groupby(['2'])['line_number'].shift(1)
lines['diff'] = lines['line_number'] - lines['line_number_shifted']
lines['diff'] = lines['diff'].fillna(1.0)

lines

,0,1,2,3,4,line_number,line_number_shifted,diff,next_movie
86,L49,u0,m0,BIANCA,Did you change your hair?,49,NaN,1.0,NaN
85,L50,u3,m0,CHASTITY,No.,50,49.0,1.0,m0
84,L51,u0,m0,BIANCA,You might wanna think about it,51,50.0,1.0,m0
648,L59,u9,m0,PATRICK,I missed you.,59,51.0,8.0,m0
647,L60,u8,m0,MISS PERKY,It says here you exposed yourself to a group o...,60,59.0,1.0,m0
...,...,...,...,...,...,...,...,...,...
48408,L300698,u1463,m99,INDIANA,The last Sankara Stone.,300698,300691.0,7.0,m99
48407,L300699,u1468,m99,WILLIE,And they don't even know what it really is.,300699,300698.0,1.0,m99
48406,L300700,u1463,m99,INDIANA,Well you didn't get your prince and there goes...,300700,300699.0,1.0,m99
48405,L300701,u1468,m99,WILLIE,You didn't do so well yourself. Finding that s...,300701,300700.0,1.0,m99


In [155]:
#creating the conversation id variable
lines['new_section'] = lines['diff'].where(lines['diff'] != 1).cumsum()
lines['new_movie'] = lines['next_movie'].where(lines['2'] != lines['next_movie'])


id = 0 
def increment(r):
    global id
    if isna(r.new_section):
        id += 1
        return(id)
    else: 
        if isna(r.new_movie):
            id += 1
            return(id)
        else:
            return(id)

for i in range(0, len(lines)):
    r = lines[i,]


SyntaxError: invalid syntax (1446146604.py, line 9)

In [117]:
conversations['conv_list'] = [x.replace('[', '').replace(']', '').replace("'", '').split(' ') for x in conversations['3']]
conversations['dialog_id'] = conversations['2'] + conversations['3']

In [ ]:
lines['line_number'] = 

In [119]:
conversations.conv_list.apply(pd.Series) \
    .merge(conversations, right_index = True, left_index = True) \
        .drop(['conv_list'], axis = 1) \
            .melt(id_vars = ['dialog_id'], value_name = 'lines') \
                .drop('variable', axis = 1) \
                    .dropna()

,dialog_id,lines
0,m0['L194' 'L195' 'L196' 'L197'],L194
1,m0['L198' 'L199'],L198
2,m0['L200' 'L201' 'L202' 'L203'],L200
3,m0['L204' 'L205' 'L206'],L204
4,m0['L207' 'L208'],L207
...,...,...
7728016,m616['L666324' 'L666325' 'L666326' 'L666327'],['L666324' 'L666325' 'L666326' 'L666327']
7728017,m616['L666575' 'L666576'],['L666575' 'L666576']
7728018,m616['L666256' 'L666257'],['L666256' 'L666257']
7728019,m616['L666369' 'L666370' 'L666371' 'L666372'],['L666369' 'L666370' 'L666371' 'L666372']


In [6]:
pd.DataFrame([lines[0], lines[1]], columns = ['line_number', 'speaker', 'movie'])

,0,1,2,3,4
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
